# Implement the data Processing , Data Visualization and Data Wrangling on any real-world problem i.e Covid_19 dataset to view the active cases on the world Map using Choropleth and also plot the cases.

Visualization of Choropleth map to show the details of COVID-19 cases for Indian States and Union Territories.It displays- total cases, active cases, 
and total deaths in each Indian State. Try to render a map that is as interactive as possible.

- Geospatial Data Visualization and analysis using Folium library.
- COVID-19 data is accessed through an API.
- Coordinates for Indian States are traced using Geo.JSON.

In [ ]:
Folium build on the data wrangling strengths of the Python ecosystem and the mapping strengths of the leafiet.js library. 
    
Folium makes it easy to visualize data that is been manipulated in Python on an interactive leafiet map. It enables both the binding of data to a 
map for choropleth visuallizations as well as passing rich vector/raster/HTML visualizations as markers on the map. A chorpleth map is a themetic map 
in which areas are shaded or patterned in proportion to the measurement of the stattiscal varible being displayed on the map such as population 
density or per capita income.

The library has a number of built-in tiesets from OpenStreetMap, Mapbox, and Stamen and supports custom tiesets with Mapbox or Cloudmade API keys,
folium supports both Image, VIdeo, GeoJSON and TopoJSON overdays.

# Import required modules/libraries.

In [1]:
pip install folium

In [ ]:
import json- This imports the built-in "json" module for working with JSON data.

import requests- This imports the "requests" library for making HTTP requests and interacting with APIs.

In [3]:
import folium as flm
from folium import plugins
import pandas as pd
import json
import requests

# Loading Geo-JSON data

In [ ]:
This data consists of co-ordinates which is used to plot the map

In [6]:
with open(r'Indian_States.json') as f:
    geojson_states = json.load(f)

# Assigning name of states to a key called 'id'

In [8]:
for i in geojson_states['features']:
    i['id']=i['properties']['NAME_1']

In [10]:
# STEP-1: Load Covid-19 Data from Local JSON file
with open("data.json","r") as file:
    covid_current = json.load(file)

In [14]:
# STEP-2: Convert JSON Data to Pandas Dataframe
"""Instead of using the deprecated .append() method, we create a list of dicyionaries  to store state-wise COVID-19 Data.
The first entry in the JSON (total cases for the entire country) is skipped.
We only keep data for individual states, exclusing "State Unassigned".
Finally, the list is converted into a Pandas DataFrame for Further Processing."""

import pandas as pd
# Create a list of dictionaries instead of using append()
data_list=[]
for state_data in covid_current["statewise"][1:]: 
    if state_data["state"] != "State Unassigned":
        data_list.append({
            "State": state_data["state"],
            "Total Cases": int(state_data["confirmed"]),
            "Active Cases": int(state_data["active"]),
            "Deaths": int(state_data["deaths"])
        })
# Convert the list into a Dataframe
df_covid = pd.DataFrame(data_list)

In [16]:
df_covid.rename(columns={
    "Total Cases": "total_case",
    "Active Cases": "active_case",
    "Deaths": "total_deaths"
}, inplace=True)

In [ ]:
# STEP-3: Verify and Correct State Name Mismathches
"""This Step ensures that all state names in the dataset match those in the GeoJSON file:
1. Extract the list of State names from the GeoJSON file(NAME_1 property).
2. Extract the list of State names from the COVID-19 DataFrame.
3. Identifies state that are present in GeoJSON but missing in df_covid.
4. Manually corrects known mismatches (e.g;, "Dadra and Nagra HAveli and Daman and Diu" is renamed to "Dadra and NAgra Haveli" to ensure consistency).
This correction ensures that every state is correctly mapped to its respective COVID-19 data when visualized on the chlorpleth map."""

#Get list of states from GeoJSON 
geojson_states_names = [feature["properties"]["NAME_1"] for feature in geojson_states["features"]]

#Get list of states from Dataframe
df_covid_states = df_covid{"State"].unique()

#Find missing data
missing_states = set(geojson_states_names) - set(df_covid_states)
print("States present in GeoJSON but missing in DataFrame:", missing_states)

#Fix known mismatches manually 
state_name_correction = {
    "Dadra and Nagra Haveli and Daman and Diu" : "Dadra and NAgra Haveli"